# Energy-based Pointing Game

Cell directly below:
- iterates one model (e.g. best performing fold 1 model of B-Cos ResNet50)
- determines Energy-based pointing game of the fold fitting to the model (validation set) - first fold model focuses on validation set of first fold
- calculates average energy-based pointing game result of the validation set
- Results: higher proportion (result of energy-based pointing game) on correctly classified images

Second Cell below:
- prints image with corresponding explanation image
- executes Energy-based pointing game on one image - giving results

# Calculate EPG for Positive Contributions
value rather low

In [2]:
from libraries.energyPointGame import energy_point_game
from libraries.ImageUtil import find_original_image_name
from libraries.bcosconv2d import NormedConv2d


import random
import numpy as np
import torch
import pandas as pd
import pydicom
import os
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt
import pickle
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms

'''from dataset.pneumonia_dataset import PneumoniaDataset'''
from dataset.augmentations import no_augmentations
from libraries.bcosconv2d import NormedConv2d
#from blurpool.blur_bcosconv2d import ModifiedBcosConv2d
from pooling.flc_bcosconv2d import ModifiedFLCBcosConv2d
from libraries.bcoslinear import BcosLinear

np.random.seed(0)
random.seed(0)
torch.manual_seed(0)

original_width, original_height = 1024, 1024
explanation_width, explanation_height = 224, 224

image_folder = r"C:\Users\Admin\Documents\rsna-pneumonia-detection-challenge\stage_2_train_images"
#model_path = r"C:\Users\Admin\Documents\MasterThesis\results\Transformer_Conv_BCos\no_nosamp\seed_0\pneumonia_detection_model_transformer_bcos_bestf1_1.pth"
# model_path = r"C:\Users\Admin\Documents\MasterThesis\results\Pneumonia\ResNet50_BCos\light_oversamp\seed_0\pneumonia_detection_model_resnet_bestf1_1.pth"
model_path = r"C:\Users\Admin\Downloads\BCosResNet50_test\seed_0_1\pneumonia_detection_model_resnet_bestf1_1.pth"
csv_path_splits = r"G:\Meine Ablage\Universität\Master Thesis\Pneumonia\training\grouped_data.csv"
csv_path = r"C:\Users\Admin\Documents\rsna-pneumonia-detection-challenge\stage_2_train_labels.csv"
splits_path = r"G:\Meine Ablage\Universität\Master Thesis\Pneumonia\training\splits\splits_balanced_fix.pkl"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#model = torch.hub.load('B-cos/B-cos-v2', 'vitc_b_patch1_14', pretrained=True)
#model[0].linear_head.linear = BcosLinear(in_features=768, out_features=2, bias=False, b=2)

model = torch.hub.load('B-cos/B-cos-v2', 'resnet50', pretrained=True)
model.fc.linear = NormedConv2d(2048, 2, kernel_size=(1, 1), stride=(1, 1), bias=False) # code from B-cos paper reused to adjust network

# Modify model and load model
'''model = torch.hub.load('B-cos/B-cos-v2', 'resnet50', pretrained=True)

model.layer2[0].conv2 = ModifiedFLCBcosConv2d(128, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), b=2, transpose=True)
model.layer2[0].downsample[0] = ModifiedFLCBcosConv2d(256, 512, kernel_size=(1, 1), stride=(2, 2), b=2, transpose=False)

model.layer3[0].conv2 = ModifiedFLCBcosConv2d(256, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), b=2, transpose=True)
model.layer3[0].downsample[0] = ModifiedFLCBcosConv2d(512, 1024, kernel_size=(1, 1), stride=(2, 2), b=2, transpose=False)

model.layer4[0].conv2 = ModifiedFLCBcosConv2d(512, 512, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), b=2, transpose=True)
model.layer4[0].downsample[0] = ModifiedFLCBcosConv2d(1024, 2048, kernel_size=(1, 1), stride=(2, 2), b=2, transpose=False)        
model.fc.linear = NormedConv2d(2048, 2, kernel_size=(1, 1), stride=(1, 1), bias=False) # code from B-cos paper reused to adjust network
'''
state_dict = torch.load(model_path)
model.load_state_dict(state_dict)

model.to(device)


data = pd.read_csv(csv_path)
data_splits = pd.read_csv(csv_path_splits)

with open(splits_path, 'rb') as f:
    splits = pickle.load(f)

# Loop over whole validation set of first fold 
first_split = splits[0] # fold selection
val_idx = first_split[1]  # Only use the validation indices from the first fold
val_data = data_splits.iloc[val_idx]



def energy_point_game_debug(coords, contrib_map):
    num_positive = (contrib_map > 0).sum().item()
    num_negative = (contrib_map < 0).sum().item()
    
    print(f"Bounding Box Coordinates: {coords.tolist()}")
    print(f"Contribution Map Min/Max: {contrib_map.min().item()}, {contrib_map.max().item()}")
    print(f"Positive Values Count: {num_positive}, Negative Values Count: {num_negative}")
    
    result = energy_point_game(coords, contrib_map)
    print(f"Energy Point Game Score: {result}\n")
    
    return result

### alternative in new models
class PneumoniaDataset(Dataset):
    def __init__(self, dataframe, image_folder, transform=None):
        self.data = dataframe
        self.image_folder = image_folder
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        image_path = os.path.join(self.image_folder, f"{row['patientId']}.dcm")
        label = row['Target']
        patient_id = row['patientId']

        # Load DICOM file and process it into RGB format
        dicom = pydicom.dcmread(image_path)
        image = dicom.pixel_array
        image = Image.fromarray(image).convert("RGB")
        
        if self.transform:
            image = self.transform(image)

        return image, torch.tensor(label, dtype=torch.long), patient_id


transform = transforms.Compose([
    transforms.ToTensor()# Normalize with ImageNet stats
])


''' 
transform = no_augmentations() 
val_dataset = PneumoniaDataset(val_data, image_folder, transform=transform)

'''
val_dataset = PneumoniaDataset(val_data, image_folder, transform=transform)

val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

scale_x = explanation_width / original_width
scale_y = explanation_height / original_height

proportions = []
proportions_correct = []
proportions_incorrect = []
count_correct = 0
count_incorrect = 0
model.eval()
with torch.no_grad():
    for images, labels, patient_ids in val_loader:
        #images, labels = images.to(device), labels.to(device)
        labels = labels.to(device)
        six_channel_images = []
        for img_tensor in images:
            numpy_image = (img_tensor.permute(1, 2, 0).cpu().numpy() * 255).astype(np.uint8)
            pil_image = Image.fromarray(numpy_image)
            transformed_image = model.transform(pil_image)
            six_channel_images.append(transformed_image)
            
        six_channel_images = torch.stack(six_channel_images).to(device)
        
        for image, label, patient_id in zip(six_channel_images, labels, patient_ids):
            filtered_rows = data[(data['patientId'] == patient_id) & (data['Target'] == 1)]
            if not filtered_rows.empty: 
                image = image[None]
                output = model(image)
                #print(image.shape)
                prediction = torch.argmax(output, dim=1)
                expl = model.explain(image)
                #prediction = expl['prediction']
                contribution_map = expl['contribution_map'].squeeze(0).cpu()
                #contribution_map = torch.clamp(contribution_map, min=0)
                contribution_map[contribution_map<0] = 0  
                proportion = 0.0
                for _, row in filtered_rows.iterrows():
                    x, y, width, height = round(row["x"] * scale_x), round(row["y"] * scale_y), round(row["width"] * scale_x), round(row["height"] * scale_y)
                    #coordinates_list = [y, x, y + height,x + width]
                    coordinates_list = [x, y, x + width, y + height]
                    coordinates_tensor = torch.tensor(coordinates_list, dtype=torch.int32)
                    ebpg_result = energy_point_game(coordinates_tensor, contribution_map)
                    proportion += ebpg_result
                    
                proportions.append(proportion)
                if prediction == 1:
                    proportions_correct.append(proportion)
                    count_correct = count_correct + 1
                else:
                    proportions_incorrect.append(proportion)
                    count_incorrect = count_incorrect + 1
                

if proportions:
    avg_proportion = sum(proportions) / len(proportions)
    avg_proportion_incorrect = sum(proportions_incorrect) / len(proportions_incorrect)
    avg_proportion_correct = sum(proportions_correct) / len(proportions_correct)
    
    avg_proportion = round(avg_proportion.item(), 4)
    avg_proportion_incorrect = round(avg_proportion_incorrect.item(), 4)
    avg_proportion_correct = round(avg_proportion_correct.item(), 4)

    print(f"Average Energy-Based Pointing Game Proportion: {avg_proportion}")
    print(f"Average Energy-Based Pointing Game Proportion of Incorrectly Classified Images: {avg_proportion_incorrect}, Count: {count_incorrect}")
    print(f"Average Energy-Based Pointing Game Proportion of Correctly Classified Images: {avg_proportion_correct}, Count: {count_correct}")

else:
    print("No valid proportions were found.")





Using cache found in C:\Users\Admin/.cache\torch\hub\B-cos_B-cos-v2_main
C:\Users\Admin/.cache\torch\hub\B-cos_B-cos-v2_main\bcos\common.py:152: UserWarning: Input tensor did not require grad! Has been set automatically to True!
  warnings.warn(


Average Energy-Based Pointing Game Proportion: 0.1046
Average Energy-Based Pointing Game Proportion of Incorrectly Classified Images: 0.0785, Count: 412
Average Energy-Based Pointing Game Proportion of Correctly Classified Images: 0.1182, Count: 790


# Calculate EPG for Negative Contributions - everything aside Bounding Boxes

In [ ]:
from libraries.energyPointGame import energy_point_game
from libraries.ImageUtil import find_original_image_name
from libraries.bcosconv2d import NormedConv2d


import random
import numpy as np
import torch
import pandas as pd
import pydicom
import os
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt
import pickle
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms

'''from dataset.pneumonia_dataset import PneumoniaDataset'''
from dataset.augmentations import no_augmentations
from libraries.bcosconv2d import NormedConv2d
#from blurpool.blur_bcosconv2d import ModifiedBcosConv2d
from pooling.flc_bcosconv2d import ModifiedFLCBcosConv2d
from libraries.bcoslinear import BcosLinear

np.random.seed(0)
random.seed(0)
torch.manual_seed(0)

original_width, original_height = 1024, 1024
explanation_width, explanation_height = 224, 224

image_folder = r"C:\Users\Admin\Documents\rsna-pneumonia-detection-challenge\stage_2_train_images"
#model_path = r"C:\Users\Admin\Documents\MasterThesis\results\Transformer_Conv_BCos\no_nosamp\seed_0\pneumonia_detection_model_transformer_bcos_bestf1_1.pth"
model_path = r"C:\Users\Admin\Documents\MasterThesis\results\ResNet50_BCos\light_oversamp_nonorm\seed_0\pneumonia_detection_model_resnet_bcos_bestf1_1.pth"
csv_path_splits = r"G:\Meine Ablage\Universität\Master Thesis\Pneumonia\training\grouped_data.csv"
csv_path = r"C:\Users\Admin\Documents\rsna-pneumonia-detection-challenge\stage_2_train_labels.csv"
splits_path = r"G:\Meine Ablage\Universität\Master Thesis\Pneumonia\training\splits\splits_balanced_fix.pkl"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#model = torch.hub.load('B-cos/B-cos-v2', 'vitc_b_patch1_14', pretrained=True)
#model[0].linear_head.linear = BcosLinear(in_features=768, out_features=2, bias=False, b=2)

model = torch.hub.load('B-cos/B-cos-v2', 'resnet50', pretrained=True)
model.fc.linear = NormedConv2d(2048, 2, kernel_size=(1, 1), stride=(1, 1), bias=False) # code from B-cos paper reused to adjust network

# Modify model and load model
'''model = torch.hub.load('B-cos/B-cos-v2', 'resnet50', pretrained=True)

model.layer2[0].conv2 = ModifiedFLCBcosConv2d(128, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), b=2, transpose=True)
model.layer2[0].downsample[0] = ModifiedFLCBcosConv2d(256, 512, kernel_size=(1, 1), stride=(2, 2), b=2, transpose=False)

model.layer3[0].conv2 = ModifiedFLCBcosConv2d(256, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), b=2, transpose=True)
model.layer3[0].downsample[0] = ModifiedFLCBcosConv2d(512, 1024, kernel_size=(1, 1), stride=(2, 2), b=2, transpose=False)

model.layer4[0].conv2 = ModifiedFLCBcosConv2d(512, 512, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), b=2, transpose=True)
model.layer4[0].downsample[0] = ModifiedFLCBcosConv2d(1024, 2048, kernel_size=(1, 1), stride=(2, 2), b=2, transpose=False)        
model.fc.linear = NormedConv2d(2048, 2, kernel_size=(1, 1), stride=(1, 1), bias=False) # code from B-cos paper reused to adjust network
'''
state_dict = torch.load(model_path)
model.load_state_dict(state_dict)

model.to(device)


data = pd.read_csv(csv_path)
data_splits = pd.read_csv(csv_path_splits)

with open(splits_path, 'rb') as f:
    splits = pickle.load(f)

# Loop over whole validation set of first fold 
first_split = splits[0] # fold selection
val_idx = first_split[1]  # Only use the validation indices from the first fold
val_data = data_splits.iloc[val_idx]



def energy_point_game_debug(coords, contrib_map):
    num_positive = (contrib_map > 0).sum().item()
    num_negative = (contrib_map < 0).sum().item()
    
    print(f"Bounding Box Coordinates: {coords.tolist()}")
    print(f"Contribution Map Min/Max: {contrib_map.min().item()}, {contrib_map.max().item()}")
    print(f"Positive Values Count: {num_positive}, Negative Values Count: {num_negative}")
    
    result = energy_point_game(coords, contrib_map)
    print(f"Energy Point Game Score: {result}\n")
    
    return result

### alternative in new models
class PneumoniaDataset(Dataset):
    def __init__(self, dataframe, image_folder, transform=None):
        self.data = dataframe
        self.image_folder = image_folder
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        image_path = os.path.join(self.image_folder, f"{row['patientId']}.dcm")
        label = row['Target']
        patient_id = row['patientId']

        # Load DICOM file and process it into RGB format
        dicom = pydicom.dcmread(image_path)
        image = dicom.pixel_array
        image = Image.fromarray(image).convert("RGB")
        
        if self.transform:
            image = self.transform(image)

        return image, torch.tensor(label, dtype=torch.long), patient_id


transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # Normalize with ImageNet stats
])


''' 
transform = no_augmentations() 
val_dataset = PneumoniaDataset(val_data, image_folder, transform=transform)

'''
val_dataset = PneumoniaDataset(val_data, image_folder, transform=transform)

val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

scale_x = explanation_width / original_width
scale_y = explanation_height / original_height

proportions = []
proportions_correct = []
proportions_incorrect = []
count_correct = 0
count_incorrect = 0
model.eval()
with torch.no_grad():
    for images, labels, patient_ids in val_loader:
        #images, labels = images.to(device), labels.to(device)
        labels = labels.to(device)
        six_channel_images = []
        for img_tensor in images:
            numpy_image = (img_tensor.permute(1, 2, 0).cpu().numpy() * 255).astype(np.uint8)
            pil_image = Image.fromarray(numpy_image)
            transformed_image = model.transform(pil_image)
            six_channel_images.append(transformed_image)
            
        six_channel_images = torch.stack(six_channel_images).to(device)
        
        for image, label, patient_id in zip(six_channel_images, labels, patient_ids):
            filtered_rows = data[(data['patientId'] == patient_id) & (data['Target'] == 1)]
            if not filtered_rows.empty: 
                image = image[None]
                output = model(image)
                #print(image.shape)
                prediction = torch.argmax(output, dim=1)
                expl = model.explain(image)
                #prediction = expl['prediction']
                contribution_map = expl['contribution_map'].squeeze(0).cpu()
                contribution_map[contribution_map>0] = 0  
                proportion = 1.0
                for _, row in filtered_rows.iterrows():
                    x, y, width, height = round(row["x"] * scale_x), round(row["y"] * scale_y), round(row["width"] * scale_x), round(row["height"] * scale_y)
                    coordinates_list = [x, y, x + width, y + height]
                    coordinates_tensor = torch.tensor(coordinates_list, dtype=torch.int32)
                    ebpg_result = energy_point_game(coordinates_tensor, contribution_map)
                    proportion = proportion - ebpg_result
                    
                proportions.append(proportion)                

if proportions:
    avg_proportion = sum(proportions) / len(proportions)    
    avg_proportion = round(avg_proportion.item(), 4)
    print(f"Average Energy-Based Pointing Game Proportion: {avg_proportion}")

else:
    print("No valid proportions were found.")


Using cache found in C:\Users\Admin/.cache\torch\hub\B-cos_B-cos-v2_main
C:\Users\Admin/.cache\torch\hub\B-cos_B-cos-v2_main\bcos\common.py:152: UserWarning: Input tensor did not require grad! Has been set automatically to True!
  warnings.warn(


Average Energy-Based Pointing Game Proportion: 0.8687


# Calculate EPG across all folds

In [ ]:

from libraries.energyPointGame import energy_point_game, energy_point_game_recall
from libraries.bcosconv2d import NormedConv2d

import random
import numpy as np
import torch
import pandas as pd
import pydicom
import os
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt
import pickle
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from libraries.bcosconv2d import NormedConv2d
#from blurpool.blur_bcosconv2d import ModifiedBcosConv2d
from pooling.flc_bcosconv2d import ModifiedFLCBcosConv2d
from libraries.bcoslinear import BcosLinear

np.random.seed(0)
random.seed(0)
torch.manual_seed(0)

original_width, original_height = 1024, 1024
explanation_width, explanation_height = 224, 224

image_folder = r"C:\Users\Admin\Documents\rsna-pneumonia-detection-challenge\stage_2_train_images"
csv_path_splits = r"G:\Meine Ablage\Universität\Master Thesis\Pneumonia\training\grouped_data.csv"
csv_path = r"C:\Users\Admin\Documents\rsna-pneumonia-detection-challenge\stage_2_train_labels.csv"
splits_path = r"G:\Meine Ablage\Universität\Master Thesis\Pneumonia\training\splits\splits_balanced_fix.pkl"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

data = pd.read_csv(csv_path)
data_splits = pd.read_csv(csv_path_splits)

with open(splits_path, 'rb') as f:
    splits = pickle.load(f)

# Loop over whole validation set of first fold 

### alternative in new models
class PneumoniaDataset(Dataset):
    def __init__(self, dataframe, image_folder, transform=None):
        self.data = dataframe
        self.image_folder = image_folder
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        image_path = os.path.join(self.image_folder, f"{row['patientId']}.dcm")
        label = row['Target']
        patient_id = row['patientId']

        # Load DICOM file and process it into RGB format
        dicom = pydicom.dcmread(image_path)
        image = dicom.pixel_array
        image = Image.fromarray(image).convert("RGB")
        
        if self.transform:
            image = self.transform(image)

        return image, torch.tensor(label, dtype=torch.long), patient_id


transform = transforms.Compose([
    transforms.ToTensor()  # Normalize with ImageNet stats
])


''' 
transform = no_augmentations() 
val_dataset = PneumoniaDataset(val_data, image_folder, transform=transform)

'''

scale_x = explanation_width / original_width
scale_y = explanation_height / original_height

avg_proportions = []
avg_proportions_incorrect = []
avg_proportions_correct = []

#    model_path = f"C:/Users/Admin/Documents/MasterThesis/results/Pneumonia/ResNet50_FLC/no_nosamp/seed_0/pneumonia_detection_model_resnet_bestf1_{fold}.pth"

for fold in range(1,6):
    model_path = f"C:/Users/Admin/Documents/MasterThesis/results/Pneumonia/ResNet50_FLC/no_nosamp/seed_1/pneumonia_detection_model_resnet_bestf1_{fold}.pth"
    split = splits[fold-1] # fold selection
    val_idx = split[1]  # Only use the validation indices from the first fold
    val_data = data_splits.iloc[val_idx]
    val_dataset = PneumoniaDataset(val_data, image_folder, transform=transform)
    val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)


    proportions = []
    proportions_correct = []
    proportions_incorrect = []
    count_correct = 0
    count_incorrect = 0
    
    model = torch.hub.load('B-cos/B-cos-v2', 'resnet50', pretrained=True)

    model.layer2[0].conv2 = ModifiedFLCBcosConv2d(128, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), b=2, transpose=True)
    model.layer2[0].downsample[0] = ModifiedFLCBcosConv2d(256, 512, kernel_size=(1, 1), stride=(2, 2), b=2, transpose=False)

    model.layer3[0].conv2 = ModifiedFLCBcosConv2d(256, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), b=2, transpose=True)
    model.layer3[0].downsample[0] = ModifiedFLCBcosConv2d(512, 1024, kernel_size=(1, 1), stride=(2, 2), b=2, transpose=False)

    model.layer4[0].conv2 = ModifiedFLCBcosConv2d(512, 512, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), b=2, transpose=True)
    model.layer4[0].downsample[0] = ModifiedFLCBcosConv2d(1024, 2048, kernel_size=(1, 1), stride=(2, 2), b=2, transpose=False)    
    model.fc.linear = NormedConv2d(2048, 2, kernel_size=(1, 1), stride=(1, 1), bias=False) # code from B-cos paper reused to adjust network


    state_dict = torch.load(model_path)
    model.load_state_dict(state_dict)
    model.to(device)

    model.eval()
    with torch.no_grad():
        for images, labels, patient_ids in val_loader:
            #images, labels = images.to(device), labels.to(device)
            labels = labels.to(device)
            six_channel_images = []
            for img_tensor in images:
                numpy_image = (img_tensor.permute(1, 2, 0).cpu().numpy() * 255).astype(np.uint8)
                pil_image = Image.fromarray(numpy_image)
                transformed_image = model.transform(pil_image)
                six_channel_images.append(transformed_image)
                
            six_channel_images = torch.stack(six_channel_images).to(device)
            
            
            for image, label, patient_id in zip(six_channel_images, labels, patient_ids):
                filtered_rows = data[(data['patientId'] == patient_id) & (data['Target'] == 1)]
                if not filtered_rows.empty: 
                    image = image[None]
                    output = model(image)
                    #prediction = torch.argmax(output, dim=1)
                    expl = model.explain(image)
                    prediction = expl['prediction']
                    contribution_map = expl['contribution_map'].squeeze(0).cpu()
                    contribution_map[contribution_map<0] = 0  
                    proportion = 0.0
                    for _, row in filtered_rows.iterrows():
                        x, y, width, height = round(row["x"] * scale_x), round(row["y"] * scale_y), round(row["width"] * scale_x), round(row["height"] * scale_y)
                        coordinates_list = [x, y, x + width, y + height]
                        coordinates_tensor = torch.tensor(coordinates_list, dtype=torch.int32)
                        ebpg_result = energy_point_game(coordinates_tensor, contribution_map)
                        proportion += ebpg_result
                        
                    proportions.append(proportion)
                    if prediction == 1:
                        proportions_correct.append(proportion)
                        count_correct = count_correct + 1
                    else:
                        proportions_incorrect.append(proportion)
                        count_incorrect = count_incorrect + 1
    if proportions:
        avg_proportion = sum(proportions) / len(proportions)
        avg_proportion_incorrect = sum(proportions_incorrect) / len(proportions_incorrect)
        avg_proportion_correct = sum(proportions_correct) / len(proportions_correct)
        avg_proportions.append(avg_proportion)
        avg_proportions_incorrect.append(avg_proportion_incorrect)
        avg_proportions_correct.append(avg_proportion_correct)

    avg_proportion = round(avg_proportion.item(), 4)
    avg_proportion_incorrect = round(avg_proportion_incorrect.item(), 4)
    avg_proportion_correct = round(avg_proportion_correct.item(), 4)

    print(f"Average Energy-Based Pointing Game Proportion (Positive): {avg_proportion}")
    print(f"Average Energy-Based Pointing Game Proportion (Positive) of Incorrectly Classified Images: {avg_proportion_incorrect}, Count: {count_incorrect}", flush=True)
    print(f"Average Energy-Based Pointing Game Proportion (Positive) of Correctly Classified Images: {avg_proportion_correct}, Count: {count_correct}", flush=True)


final_avg_prop = sum(avg_proportions) / len(avg_proportions) 
final_avg_prop_incorrect = sum(avg_proportions_incorrect) / len(avg_proportions_incorrect)
final_avg_prop_correct = sum(avg_proportions_correct) / len(avg_proportions_correct)

final_avg_prop = round(final_avg_prop.item(), 4)
final_avg_prop_incorrect = round(final_avg_prop_incorrect.item(), 4)
final_avg_prop_correct = round(final_avg_prop_correct.item(), 4)
print()
print(f"Average Energy-Based Pointing Game Proportion (Positive) over all folds: {final_avg_prop}", flush=True)
print(f"Average Energy-Based Pointing Game Proportion (Positive) of Incorrectly Classified Images over all folds: {final_avg_prop_incorrect}", flush=True)
print(f"Average Energy-Based Pointing Game Proportion (Positive) of Correctly Classified Images over all folds: {final_avg_prop_correct}", flush=True)





Using cache found in C:\Users\Admin/.cache\torch\hub\B-cos_B-cos-v2_main
C:\Users\Admin/.cache\torch\hub\B-cos_B-cos-v2_main\bcos\common.py:152: UserWarning: Input tensor did not require grad! Has been set automatically to True!
  warnings.warn(


Average Energy-Based Pointing Game Proportion (Positive): 0.2825
Average Energy-Based Pointing Game Proportion (Positive) of Incorrectly Classified Images: 0.1217, Count: 450
Average Energy-Based Pointing Game Proportion (Positive) of Correctly Classified Images: 0.3788, Count: 752


Using cache found in C:\Users\Admin/.cache\torch\hub\B-cos_B-cos-v2_main


Average Energy-Based Pointing Game Proportion (Positive): 0.2689
Average Energy-Based Pointing Game Proportion (Positive) of Incorrectly Classified Images: 0.1505, Count: 487
Average Energy-Based Pointing Game Proportion (Positive) of Correctly Classified Images: 0.3495, Count: 715


Using cache found in C:\Users\Admin/.cache\torch\hub\B-cos_B-cos-v2_main


Average Energy-Based Pointing Game Proportion (Positive): 0.3416
Average Energy-Based Pointing Game Proportion (Positive) of Incorrectly Classified Images: 0.0977, Count: 455
Average Energy-Based Pointing Game Proportion (Positive) of Correctly Classified Images: 0.4901, Count: 748


Using cache found in C:\Users\Admin/.cache\torch\hub\B-cos_B-cos-v2_main


Average Energy-Based Pointing Game Proportion (Positive): 0.2952
Average Energy-Based Pointing Game Proportion (Positive) of Incorrectly Classified Images: 0.0241, Count: 443
Average Energy-Based Pointing Game Proportion (Positive) of Correctly Classified Images: 0.4532, Count: 760


Using cache found in C:\Users\Admin/.cache\torch\hub\B-cos_B-cos-v2_main


Average Energy-Based Pointing Game Proportion (Positive): 0.3028
Average Energy-Based Pointing Game Proportion (Positive) of Incorrectly Classified Images: 0.1217, Count: 328
Average Energy-Based Pointing Game Proportion (Positive) of Correctly Classified Images: 0.3708, Count: 874

Average Energy-Based Pointing Game Proportion (Positive) over all folds: 0.2982
Average Energy-Based Pointing Game Proportion (Positive) of Incorrectly Classified Images over all folds: 0.1032
Average Energy-Based Pointing Game Proportion (Positive) of Correctly Classified Images over all folds: 0.4085


# Calculate EPG (Negative) outside of Bounding Boxes across all folds

In [5]:
from libraries.energyPointGame import energy_point_game
from libraries.ImageUtil import find_original_image_name
from libraries.bcosconv2d import NormedConv2d


import random
import numpy as np
import torch
import pandas as pd
import pydicom
import os
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt
import pickle
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms

'''from dataset.pneumonia_dataset import PneumoniaDataset'''
from dataset.augmentations import no_augmentations
from libraries.bcosconv2d import NormedConv2d
#from blurpool.blur_bcosconv2d import ModifiedBcosConv2d
from pooling.flc_bcosconv2d import ModifiedFLCBcosConv2d
from libraries.bcoslinear import BcosLinear

np.random.seed(0)
random.seed(0)
torch.manual_seed(0)

original_width, original_height = 1024, 1024
explanation_width, explanation_height = 224, 224

image_folder = r"C:\Users\Admin\Documents\rsna-pneumonia-detection-challenge\stage_2_train_images"
csv_path_splits = r"G:\Meine Ablage\Universität\Master Thesis\Pneumonia\training\grouped_data.csv"
csv_path = r"C:\Users\Admin\Documents\rsna-pneumonia-detection-challenge\stage_2_train_labels.csv"
splits_path = r"G:\Meine Ablage\Universität\Master Thesis\Pneumonia\training\splits\splits_balanced_fix.pkl"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

data = pd.read_csv(csv_path)
data_splits = pd.read_csv(csv_path_splits)

with open(splits_path, 'rb') as f:
    splits = pickle.load(f)

# Loop over whole validation set of first fold 
first_split = splits[0] # fold selection
val_idx = first_split[1]  # Only use the validation indices from the first fold
val_data = data_splits.iloc[val_idx]

### alternative in new models
class PneumoniaDataset(Dataset):
    def __init__(self, dataframe, image_folder, transform=None):
        self.data = dataframe
        self.image_folder = image_folder
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        image_path = os.path.join(self.image_folder, f"{row['patientId']}.dcm")
        label = row['Target']
        patient_id = row['patientId']

        # Load DICOM file and process it into RGB format
        dicom = pydicom.dcmread(image_path)
        image = dicom.pixel_array
        image = Image.fromarray(image).convert("RGB")
        
        if self.transform:
            image = self.transform(image)

        return image, torch.tensor(label, dtype=torch.long), patient_id


transform = transforms.Compose([
    transforms.ToTensor()  # Normalize with ImageNet stats
])


''' 
transform = no_augmentations() 
val_dataset = PneumoniaDataset(val_data, image_folder, transform=transform)

'''
val_dataset = PneumoniaDataset(val_data, image_folder, transform=transform)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

scale_x = explanation_width / original_width
scale_y = explanation_height / original_height

avg_proportions = []
avg_proportions_incorrect = []
avg_proportions_correct = []

for fold in range(1,6):
    proportions = []
    proportions_correct = []
    proportions_incorrect = []
    count_correct = 0
    count_incorrect = 0
    model_path = f"C:/Users/Admin/Documents/MasterThesis/results/Pneumonia/ResNet50_BCos/light_oversamp_nonorm/seed_0/pneumonia_detection_model_resnet_bcos_bestf1_1.pth" 
    model = torch.hub.load('B-cos/B-cos-v2', 'resnet50', pretrained=True)
    model.fc.linear = NormedConv2d(2048, 2, kernel_size=(1, 1), stride=(1, 1), bias=False) # code from B-cos paper reused to adjust network
    state_dict = torch.load(model_path)
    model.load_state_dict(state_dict)
    model.to(device)

    model.eval()
    with torch.no_grad():
        for images, labels, patient_ids in val_loader:
            #images, labels = images.to(device), labels.to(device)
            labels = labels.to(device)
            six_channel_images = []
            for img_tensor in images:
                numpy_image = (img_tensor.permute(1, 2, 0).cpu().numpy() * 255).astype(np.uint8)
                pil_image = Image.fromarray(numpy_image)
                transformed_image = model.transform(pil_image)
                six_channel_images.append(transformed_image)
                
            six_channel_images = torch.stack(six_channel_images).to(device)
            
            for image, label, patient_id in zip(six_channel_images, labels, patient_ids):
                filtered_rows = data[(data['patientId'] == patient_id) & (data['Target'] == 1)]
                if not filtered_rows.empty: 
                    image = image[None]
                    output = model(image)
                    prediction = torch.argmax(output, dim=1)
                    expl = model.explain(image)
                    #prediction = expl['prediction']
                    contribution_map = expl['contribution_map'].squeeze(0).cpu()
                    contribution_map[contribution_map>0] = 0  
                    proportion = 1.0
                    for _, row in filtered_rows.iterrows():
                        x, y, width, height = round(row["x"] * scale_x), round(row["y"] * scale_y), round(row["width"] * scale_x), round(row["height"] * scale_y)
                        coordinates_list = [x, y, x + width, y + height]
                        coordinates_tensor = torch.tensor(coordinates_list, dtype=torch.int32)
                        ebpg_result = energy_point_game(coordinates_tensor, contribution_map)
                        proportion = proportion - ebpg_result
                        
                    proportions.append(proportion)
            if proportions:
                avg_proportion = sum(proportions) / len(proportions)
                avg_proportions.append(avg_proportion)
    
            avg_proportion = round(avg_proportion.item(), 4)

            print(f"Average Energy-Based Pointing Game Proportion (Negative): {avg_proportion}", flush=True)


    final_avg_prop = sum(avg_proportions) / len(avg_proportions)
    final_avg_prop = round(final_avg_prop.item(), 4)
    
    print(f"Average Energy-Based Pointing Game Proportion (Negative) over all folds: {final_avg_prop}", flush=True)

Using cache found in C:\Users\Admin/.cache\torch\hub\B-cos_B-cos-v2_main


FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/Admin/Documents/MasterThesis/results/Pneumonia/ResNet50_BCos/light_oversamp_nonorm/seed_0/pneumonia_detection_model_resnet_bcos_bestf1_1.pth'

# Calculate Recall across all folds



In [ ]:
from libraries.energyPointGame import energy_point_game, energy_point_game_recall
from libraries.bcosconv2d import NormedConv2d


import random
import numpy as np
import torch
import pandas as pd
import pydicom
import os
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt
import pickle
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from libraries.bcosconv2d import NormedConv2d
#from blurpool.blur_bcosconv2d import ModifiedBcosConv2d
from pooling.flc_bcosconv2d import ModifiedFLCBcosConv2d
from libraries.bcoslinear import BcosLinear


BASE_PATH = "/pfs/work7/workspace/scratch/ma_mkleinma-thesis/trained_models/"

np.random.seed(0)
random.seed(0)
torch.manual_seed(0)

original_width, original_height = 1024, 1024
explanation_width, explanation_height = 224, 224

image_folder = r"C:\Users\Admin\Documents\rsna-pneumonia-detection-challenge\stage_2_train_images"
csv_path_splits = r"G:\Meine Ablage\Universität\Master Thesis\Pneumonia\training\grouped_data.csv"
csv_path = r"C:\Users\Admin\Documents\rsna-pneumonia-detection-challenge\stage_2_train_labels.csv"
splits_path = r"G:\Meine Ablage\Universität\Master Thesis\Pneumonia\training\splits\splits_balanced_fix.pkl"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

data = pd.read_csv(csv_path)
data_splits = pd.read_csv(csv_path_splits)

with open(splits_path, 'rb') as f:
    splits = pickle.load(f)

# Loop over whole validation set of first fold 

### alternative in new models
class PneumoniaDataset(Dataset):
    def __init__(self, dataframe, image_folder, transform=None):
        self.data = dataframe
        self.image_folder = image_folder
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        image_path = os.path.join(self.image_folder, f"{row['patientId']}.dcm")
        label = row['Target']
        patient_id = row['patientId']

        # Load DICOM file and process it into RGB format
        dicom = pydicom.dcmread(image_path)
        image = dicom.pixel_array
        image = Image.fromarray(image).convert("RGB")
        
        if self.transform:
            image = self.transform(image)

        return image, torch.tensor(label, dtype=torch.long), patient_id


transform = transforms.Compose([
    transforms.ToTensor()  # Normalize with ImageNet stats
])


''' 
transform = no_augmentations() 
val_dataset = PneumoniaDataset(val_data, image_folder, transform=transform)

'''

scale_x = explanation_width / original_width
scale_y = explanation_height / original_height

avg_proportions = []
avg_proportions_incorrect = []
avg_proportions_correct = []

for fold in range(1,6):
    #model_path = f"C:/Users/Admin/Documents/MasterThesis/results/Pneumonia/Transformer_Conv_BCos/light_oversamp/seed_0/pneumonia_detection_model_transformer_bestf1_{fold}.pth"
    model_path = f"C:/Users/Admin/Documents/MasterThesis/results/Pneumonia/ResNet50_FLC/no_nosamp/seed_0/pneumonia_detection_model_resnet_bestf1_{fold}.pth"
    split = splits[fold-1] # fold selection
    val_idx = split[1]  # Only use the validation indices from the first fold
    val_data = data_splits.iloc[val_idx]
    val_dataset = PneumoniaDataset(val_data, image_folder, transform=transform)
    val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)


    proportions = []
    proportions_correct = []
    proportions_incorrect = []
    count_correct = 0
    count_incorrect = 0
    
    model = torch.hub.load('B-cos/B-cos-v2', 'resnet50', pretrained=True)

    model.layer2[0].conv2 = ModifiedFLCBcosConv2d(128, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), b=2, transpose=True)
    model.layer2[0].downsample[0] = ModifiedFLCBcosConv2d(256, 512, kernel_size=(1, 1), stride=(2, 2), b=2, transpose=False)

    model.layer3[0].conv2 = ModifiedFLCBcosConv2d(256, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), b=2, transpose=True)
    model.layer3[0].downsample[0] = ModifiedFLCBcosConv2d(512, 1024, kernel_size=(1, 1), stride=(2, 2), b=2, transpose=False)

    model.layer4[0].conv2 = ModifiedFLCBcosConv2d(512, 512, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), b=2, transpose=True)
    model.layer4[0].downsample[0] = ModifiedFLCBcosConv2d(1024, 2048, kernel_size=(1, 1), stride=(2, 2), b=2, transpose=False)    
    model.fc.linear = NormedConv2d(2048, 2, kernel_size=(1, 1), stride=(1, 1), bias=False) # code from B-cos paper reused to adjust network

    #model = torch.hub.load('B-cos/B-cos-v2', 'vitc_b_patch1_14', pretrained=True)
    #model[0].linear_head.linear = BcosLinear(in_features=768, out_features=2, bias=False, b=2)

    #model = torch.hub.load('B-cos/B-cos-v2', 'resnet50', pretrained=True)
    #model.fc.linear = NormedConv2d(2048, 2, kernel_size=(1, 1), stride=(1, 1), bias=False) # code from B-cos paper reused to adjust network

    state_dict = torch.load(model_path)
    model.load_state_dict(state_dict)
    model.to(device)

    model.eval()
    with torch.no_grad():
        for images, labels, patient_ids in val_loader:
            #images, labels = images.to(device), labels.to(device)
            labels = labels.to(device)
            six_channel_images = []
            for img_tensor in images:
                numpy_image = (img_tensor.permute(1, 2, 0).cpu().numpy() * 255).astype(np.uint8)
                pil_image = Image.fromarray(numpy_image)
                transformed_image = model.transform(pil_image)
                six_channel_images.append(transformed_image)
                
            six_channel_images = torch.stack(six_channel_images).to(device)
            
            
            for image, label, patient_id in zip(six_channel_images, labels, patient_ids):
                filtered_rows = data[(data['patientId'] == patient_id) & (data['Target'] == 1)]
                if not filtered_rows.empty: 
                    image = image[None]
                    output = model(image)
                    #prediction = torch.argmax(output, dim=1)
                    expl = model.explain(image)
                    prediction = expl['prediction']
                    contribution_map = expl['contribution_map'].squeeze(0).cpu()
                    proportion_total = 0.0
                    num_boxes = len(filtered_rows)  
                    for _, row in filtered_rows.iterrows():
                        x, y, width, height = round(row["x"] * scale_x), round(row["y"] * scale_y), round(row["width"] * scale_x), round(row["height"] * scale_y)
                        coordinates_list = [x, y, x + width, y + height]
                        coordinates_tensor = torch.tensor(coordinates_list, dtype=torch.int32)
                        ebpg_result = energy_point_game_recall(coordinates_tensor, contribution_map)
                        proportion_total += ebpg_result
                        
                    average_proportion = proportion_total / num_boxes if num_boxes > 0 else 0.0
                    proportions.append(average_proportion)
                    if prediction == 1:
                        proportions_correct.append(average_proportion)
                        count_correct = count_correct + 1
                    else:
                        proportions_incorrect.append(average_proportion)
                        count_incorrect = count_incorrect + 1
    if proportions:
        avg_proportion = sum(proportions) / len(proportions)
        avg_proportion_incorrect = sum(proportions_incorrect) / len(proportions_incorrect)
        avg_proportion_correct = sum(proportions_correct) / len(proportions_correct)
        avg_proportions.append(avg_proportion)
        avg_proportions_incorrect.append(avg_proportion_incorrect)
        avg_proportions_correct.append(avg_proportion_correct)

    avg_proportion = round(avg_proportion.item(), 4)
    avg_proportion_incorrect = round(avg_proportion_incorrect.item(), 4)
    avg_proportion_correct = round(avg_proportion_correct.item(), 4)

    print(f"Average Energy-Based Pointing Game Proportion (Positive): {avg_proportion}")
    print(f"Average Energy-Based Pointing Game Proportion (Positive) of Incorrectly Classified Images: {avg_proportion_incorrect}, Count: {count_incorrect}", flush=True)
    print(f"Average Energy-Based Pointing Game Proportion (Positive) of Correctly Classified Images: {avg_proportion_correct}, Count: {count_correct}", flush=True)


final_avg_prop = sum(avg_proportions) / len(avg_proportions) 
final_avg_prop_incorrect = sum(avg_proportions_incorrect) / len(avg_proportions_incorrect)
final_avg_prop_correct = sum(avg_proportions_correct) / len(avg_proportions_correct)

final_avg_prop = round(final_avg_prop.item(), 4)
final_avg_prop_incorrect = round(final_avg_prop_incorrect.item(), 4)
final_avg_prop_correct = round(final_avg_prop_correct.item(), 4)
print()
print(f"Average Energy-Based Pointing Game Proportion (Positive) over all folds: {final_avg_prop}", flush=True)
print(f"Average Energy-Based Pointing Game Proportion (Positive) of Incorrectly Classified Images over all folds: {final_avg_prop_incorrect}", flush=True)
print(f"Average Energy-Based Pointing Game Proportion (Positive) of Correctly Classified Images over all folds: {final_avg_prop_correct}", flush=True)




Using cache found in C:\Users\Admin/.cache\torch\hub\B-cos_B-cos-v2_main
C:\Users\Admin/.cache\torch\hub\B-cos_B-cos-v2_main\bcos\common.py:152: UserWarning: Input tensor did not require grad! Has been set automatically to True!
  warnings.warn(


Average Energy-Based Pointing Game Proportion (Positive): 0.9171
Average Energy-Based Pointing Game Proportion (Positive) of Incorrectly Classified Images: 0.7921, Count: 421
Average Energy-Based Pointing Game Proportion (Positive) of Correctly Classified Images: 0.9844, Count: 781


Using cache found in C:\Users\Admin/.cache\torch\hub\B-cos_B-cos-v2_main


Average Energy-Based Pointing Game Proportion (Positive): 0.0791
Average Energy-Based Pointing Game Proportion (Positive) of Incorrectly Classified Images: 0.0431, Count: 338
Average Energy-Based Pointing Game Proportion (Positive) of Correctly Classified Images: 0.0932, Count: 864


Using cache found in C:\Users\Admin/.cache\torch\hub\B-cos_B-cos-v2_main


Average Energy-Based Pointing Game Proportion (Positive): 0.722
Average Energy-Based Pointing Game Proportion (Positive) of Incorrectly Classified Images: 0.4514, Count: 402
Average Energy-Based Pointing Game Proportion (Positive) of Correctly Classified Images: 0.8578, Count: 801


Using cache found in C:\Users\Admin/.cache\torch\hub\B-cos_B-cos-v2_main


Average Energy-Based Pointing Game Proportion (Positive): 0.9526
Average Energy-Based Pointing Game Proportion (Positive) of Incorrectly Classified Images: 0.8734, Count: 336
Average Energy-Based Pointing Game Proportion (Positive) of Correctly Classified Images: 0.9833, Count: 867


Using cache found in C:\Users\Admin/.cache\torch\hub\B-cos_B-cos-v2_main


Average Energy-Based Pointing Game Proportion (Positive): 0.6702
Average Energy-Based Pointing Game Proportion (Positive) of Incorrectly Classified Images: 0.4494, Count: 537
Average Energy-Based Pointing Game Proportion (Positive) of Correctly Classified Images: 0.8485, Count: 665

Average Energy-Based Pointing Game Proportion (Positive) over all folds: 0.6682
Average Energy-Based Pointing Game Proportion (Positive) of Incorrectly Classified Images over all folds: 0.5219
Average Energy-Based Pointing Game Proportion (Positive) of Correctly Classified Images over all folds: 0.7535


# Calculate EPG across all seeds for 'Precision' or 'normal EPG'

In [ ]:
from libraries.energyPointGame import energy_point_game, energy_point_game_recall
from libraries.bcosconv2d import NormedConv2d

import random
import numpy as np
import torch
import pandas as pd
import pydicom
import os
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt
import pickle
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from libraries.bcosconv2d import NormedConv2d
from blurpool.blur_bcosconv2d import ModifiedBcosConv2d
from pooling.flc_bcosconv2d import ModifiedFLCBcosConv2d
from libraries.bcoslinear import BcosLinear

np.random.seed(0)
random.seed(0)
torch.manual_seed(0)

original_width, original_height = 1024, 1024
explanation_width, explanation_height = 224, 224

image_folder = r"C:\Users\Admin\Documents\rsna-pneumonia-detection-challenge\stage_2_train_images"
csv_path_splits = r"G:\Meine Ablage\Universität\Master Thesis\Pneumonia\training\grouped_data.csv"
csv_path = r"C:\Users\Admin\Documents\rsna-pneumonia-detection-challenge\stage_2_train_labels.csv"
splits_path = r"G:\Meine Ablage\Universität\Master Thesis\Pneumonia\training\splits\splits_balanced_fix.pkl"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

data = pd.read_csv(csv_path)
data_splits = pd.read_csv(csv_path_splits)

with open(splits_path, 'rb') as f:
    splits = pickle.load(f)

scale_x = explanation_width / original_width
scale_y = explanation_height / original_height



class PneumoniaDataset(Dataset):
    def __init__(self, dataframe, image_folder, transform=None):
        self.data = dataframe
        self.image_folder = image_folder
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        image_path = os.path.join(self.image_folder, f"{row['patientId']}.dcm")
        label = row['Target']
        patient_id = row['patientId']

        # Load DICOM file and process it into RGB format
        dicom = pydicom.dcmread(image_path)
        image = dicom.pixel_array
        image = Image.fromarray(image).convert("RGB")
        
        if self.transform:
            image = self.transform(image)

        return image, torch.tensor(label, dtype=torch.long), patient_id


transform = transforms.Compose([
    transforms.ToTensor()# Normalize with ImageNet stats
])

seeds = [0, 1]
all_results = []

for seed in seeds:
    print(f"\n{'='*40}")
    print(f"Processing seed {seed}")
    print(f"{'='*40}\n")
    
    avg_proportions = []
    avg_proportions_incorrect = []
    avg_proportions_correct = []

    for fold in range(1,6):
        model_path = f"C:/Users/Admin/Documents/MasterThesis/results/Pneumonia/ResNet50_BCos/light_oversamp/seed_{seed}/pneumonia_detection_model_resnet_bestf1_{fold}.pth"        
        split = splits[fold-1] # fold selection
        val_idx = split[1] 
        val_data = data_splits.iloc[val_idx]
        val_dataset = PneumoniaDataset(val_data, image_folder, transform=transform)
        val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)


        proportions = []
        proportions_correct = []
        proportions_incorrect = []
        count_correct = 0
        count_incorrect = 0
        
        model = torch.hub.load('B-cos/B-cos-v2', 'resnet50', pretrained=True)    
        model.fc.linear = NormedConv2d(2048, 2, kernel_size=(1, 1), stride=(1, 1), bias=False) # code from B-cos paper reused to adjust network


        state_dict = torch.load(model_path)
        model.load_state_dict(state_dict)
        model.to(device)

        model.eval()
        with torch.no_grad():
            for images, labels, patient_ids in val_loader:
                #images, labels = images.to(device), labels.to(device)
                labels = labels.to(device)
                six_channel_images = []
                for img_tensor in images:
                    numpy_image = (img_tensor.permute(1, 2, 0).cpu().numpy() * 255).astype(np.uint8)
                    pil_image = Image.fromarray(numpy_image)
                    transformed_image = model.transform(pil_image)
                    six_channel_images.append(transformed_image)
                    
                six_channel_images = torch.stack(six_channel_images).to(device)
                
                for image, label, patient_id in zip(six_channel_images, labels, patient_ids):
                    filtered_rows = data[(data['patientId'] == patient_id) & (data['Target'] == 1)]
                    if not filtered_rows.empty: 
                        image = image[None]
                        output = model(image)
                        #prediction = torch.argmax(output, dim=1)
                        expl = model.explain(image)
                        prediction = expl['prediction']
                        contribution_map = expl['contribution_map'].squeeze(0).cpu()
                        contribution_map[contribution_map<0] = 0  
                        proportion = 0.0
                        for _, row in filtered_rows.iterrows():
                            x, y, width, height = round(row["x"] * scale_x), round(row["y"] * scale_y), round(row["width"] * scale_x), round(row["height"] * scale_y)
                            coordinates_list = [x, y, x + width, y + height]
                            coordinates_tensor = torch.tensor(coordinates_list, dtype=torch.int32)
                            ebpg_result = energy_point_game(coordinates_tensor, contribution_map)
                            proportion += ebpg_result
                            
                        proportions.append(proportion)
                        if prediction == 1:
                            proportions_correct.append(proportion)
                            count_correct = count_correct + 1
                        else:
                            proportions_incorrect.append(proportion)
                            count_incorrect = count_incorrect + 1
        if proportions:
            avg_proportion = sum(proportions) / len(proportions)
            avg_proportion_incorrect = sum(proportions_incorrect) / len(proportions_incorrect)
            avg_proportion_correct = sum(proportions_correct) / len(proportions_correct)
            avg_proportions.append(avg_proportion)
            avg_proportions_incorrect.append(avg_proportion_incorrect)
            avg_proportions_correct.append(avg_proportion_correct)

        avg_proportion = round(avg_proportion.item(), 4)
        avg_proportion_incorrect = round(avg_proportion_incorrect.item(), 4)
        avg_proportion_correct = round(avg_proportion_correct.item(), 4)
        
        print(f"Average Energy-Based Pointing Game Proportion (Positive): {avg_proportion}")
        print(f"Average Energy-Based Pointing Game Proportion (Positive) of Incorrectly Classified Images: {avg_proportion_incorrect}, Count: {count_incorrect}", flush=True)
        print(f"Average Energy-Based Pointing Game Proportion (Positive) of Correctly Classified Images: {avg_proportion_correct}, Count: {count_correct}", flush=True)



    final_avg_prop = sum(avg_proportions) / len(avg_proportions) 
    final_avg_prop_incorrect = sum(avg_proportions_incorrect) / len(avg_proportions_incorrect)
    final_avg_prop_correct = sum(avg_proportions_correct) / len(avg_proportions_correct)

    final_avg_prop = round(final_avg_prop.item(), 4)
    final_avg_prop_incorrect = round(final_avg_prop_incorrect.item(), 4)
    final_avg_prop_correct = round(final_avg_prop_correct.item(), 4)
    print(f"Average Energy-Based Pointing Game Proportion (Positive) over all folds: {final_avg_prop}", flush=True)
    print(f"Average Energy-Based Pointing Game Proportion (Positive) of Incorrectly Classified Images over all folds: {final_avg_prop_incorrect}", flush=True)
    print(f"Average Energy-Based Pointing Game Proportion (Positive) of Correctly Classified Images over all folds: {final_avg_prop_correct}", flush=True)

        
        # Store results per fold with seed information
    fold_results = {
        'seed': seed,
        'fold': fold,
        'avg_proportion': final_avg_prop,
        'avg_incorrect': final_avg_prop_incorrect,
        'avg_correct': final_avg_prop_correct,
    }
    all_results.append(fold_results)

# Calculate aggregate statistics
def calculate_stats(values):
    return f"{np.mean(values):.4f} ± {np.std(values):.4f}"

final_proportions = [r['avg_proportion'] for r in all_results]
final_incorrect = [r['avg_incorrect'] for r in all_results]
final_correct = [r['avg_correct'] for r in all_results]

print("\n\n=== Final Aggregate Statistics ===")
print(f"Overall EPG (Positive): {calculate_stats(final_proportions)}")
print(f"EPG Incorrect Classifications: {calculate_stats(final_incorrect)}")
print(f"EPG Correct Classifications: {calculate_stats(final_correct)}")



Processing seed 0



Using cache found in C:\Users\Admin/.cache\torch\hub\B-cos_B-cos-v2_main
C:\Users\Admin/.cache\torch\hub\B-cos_B-cos-v2_main\bcos\common.py:152: UserWarning: Input tensor did not require grad! Has been set automatically to True!
  warnings.warn(


Average Energy-Based Pointing Game Proportion (Positive): 0.0924
Average Energy-Based Pointing Game Proportion (Positive) of Incorrectly Classified Images: 0.0057, Count: 296
Average Energy-Based Pointing Game Proportion (Positive) of Correctly Classified Images: 0.1207, Count: 906


Using cache found in C:\Users\Admin/.cache\torch\hub\B-cos_B-cos-v2_main


Average Energy-Based Pointing Game Proportion (Positive): 0.2301
Average Energy-Based Pointing Game Proportion (Positive) of Incorrectly Classified Images: 0.0855, Count: 290
Average Energy-Based Pointing Game Proportion (Positive) of Correctly Classified Images: 0.2761, Count: 912


...

Average Energy-Based Pointing Game Proportion (Positive): 0.0846
Average Energy-Based Pointing Game Proportion (Positive) of Incorrectly Classified Images: 0.0015, Count: 279
Average Energy-Based Pointing Game Proportion (Positive) of Correctly Classified Images: 0.1097, Count: 924


Using cache found in C:\Users\Admin/.cache\torch\hub\B-cos_B-cos-v2_main


Average Energy-Based Pointing Game Proportion (Positive): 0.171
Average Energy-Based Pointing Game Proportion (Positive) of Incorrectly Classified Images: 0.0327, Count: 240
Average Energy-Based Pointing Game Proportion (Positive) of Correctly Classified Images: 0.2055, Count: 963


Using cache found in C:\Users\Admin/.cache\torch\hub\B-cos_B-cos-v2_main


Average Energy-Based Pointing Game Proportion (Positive): 0.2279
Average Energy-Based Pointing Game Proportion (Positive) of Incorrectly Classified Images: 0.0924, Count: 318
Average Energy-Based Pointing Game Proportion (Positive) of Correctly Classified Images: 0.2767, Count: 884
Average Energy-Based Pointing Game Proportion (Positive) over all folds: 0.1226
Average Energy-Based Pointing Game Proportion (Positive) of Incorrectly Classified Images over all folds: 0.026
Average Energy-Based Pointing Game Proportion (Positive) of Correctly Classified Images over all folds: 0.1518


=== Final Aggregate Statistics ===
Overall EPG (Positive): 0.1535 ± 0.0309
EPG Incorrect Classifications: 0.0437 ± 0.0177
EPG Correct Classifications: 0.1869 ± 0.0351


# Recall-based calculation of EPG across all seeds

In [ ]:
from libraries.energyPointGame import energy_point_game, energy_point_game_recall
from libraries.bcosconv2d import NormedConv2d

import random
import numpy as np
import torch
import pandas as pd
import pydicom
import os
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt
import pickle
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from libraries.bcosconv2d import NormedConv2d
from blurpool.blur_bcosconv2d import ModifiedBcosConv2d
from pooling.flc_bcosconv2d import ModifiedFLCBcosConv2d
from libraries.bcoslinear import BcosLinear

np.random.seed(0)
random.seed(0)
torch.manual_seed(0)

original_width, original_height = 1024, 1024
explanation_width, explanation_height = 224, 224

image_folder = r"C:\Users\Admin\Documents\rsna-pneumonia-detection-challenge\stage_2_train_images"
csv_path_splits = r"G:\Meine Ablage\Universität\Master Thesis\Pneumonia\training\grouped_data.csv"
csv_path = r"C:\Users\Admin\Documents\rsna-pneumonia-detection-challenge\stage_2_train_labels.csv"
splits_path = r"G:\Meine Ablage\Universität\Master Thesis\Pneumonia\training\splits\splits_balanced_fix.pkl"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

data = pd.read_csv(csv_path)
data_splits = pd.read_csv(csv_path_splits)

with open(splits_path, 'rb') as f:
    splits = pickle.load(f)

scale_x = explanation_width / original_width
scale_y = explanation_height / original_height



class PneumoniaDataset(Dataset):
    def __init__(self, dataframe, image_folder, transform=None):
        self.data = dataframe
        self.image_folder = image_folder
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        image_path = os.path.join(self.image_folder, f"{row['patientId']}.dcm")
        label = row['Target']
        patient_id = row['patientId']

        # Load DICOM file and process it into RGB format
        dicom = pydicom.dcmread(image_path)
        image = dicom.pixel_array
        image = Image.fromarray(image).convert("RGB")
        
        if self.transform:
            image = self.transform(image)

        return image, torch.tensor(label, dtype=torch.long), patient_id


transform = transforms.Compose([
    transforms.ToTensor()# Normalize with ImageNet stats
])

seeds = [0, 1]
all_results = []

for seed in seeds:
    print(f"\n{'='*40}")
    print(f"Processing seed {seed}")
    print(f"{'='*40}\n")
    
    avg_proportions = []
    avg_proportions_incorrect = []
    avg_proportions_correct = []

    for fold in range(1,6):
        model_path = f"C:/Users/Admin/Documents/MasterThesis/results/Pneumonia/ResNet50_Blur/no_nosamp/seed_{seed}/pneumonia_detection_model_resnet_bcos_bestf1_{fold}.pth"        
        split = splits[fold-1] # fold selection
        val_idx = split[1] 
        val_data = data_splits.iloc[val_idx]
        val_dataset = PneumoniaDataset(val_data, image_folder, transform=transform)
        val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)


        proportions = []
        proportions_correct = []
        proportions_incorrect = []
        count_correct = 0
        count_incorrect = 0
        
        model = torch.hub.load('B-cos/B-cos-v2', 'resnet50', pretrained=True)
        model.layer2[0].conv2 = ModifiedBcosConv2d(128, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), b=2)
        model.layer2[0].downsample[0] = ModifiedBcosConv2d(256, 512, kernel_size=(1, 1), stride=(2, 2), b=2)

        model.layer3[0].conv2 = ModifiedBcosConv2d(256, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), b=2)
        model.layer3[0].downsample[0] = ModifiedBcosConv2d(512, 1024, kernel_size=(1, 1), stride=(2, 2), b=2)

        model.layer4[0].conv2 = ModifiedBcosConv2d(512, 512, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), b=2)
        model.layer4[0].downsample[0] = ModifiedBcosConv2d(1024, 2048, kernel_size=(1, 1), stride=(2, 2), b=2)
        model.fc.linear = NormedConv2d(2048, 2, kernel_size=(1, 1), stride=(1, 1), bias=False) # code from B-cos paper reused to adjust network


        state_dict = torch.load(model_path)
        model.load_state_dict(state_dict)
        model.to(device)

        model.eval()
        with torch.no_grad():
            for images, labels, patient_ids in val_loader:
                #images, labels = images.to(device), labels.to(device)
                labels = labels.to(device)
                six_channel_images = []
                for img_tensor in images:
                    numpy_image = (img_tensor.permute(1, 2, 0).cpu().numpy() * 255).astype(np.uint8)
                    pil_image = Image.fromarray(numpy_image)
                    transformed_image = model.transform(pil_image)
                    six_channel_images.append(transformed_image)
                    
                six_channel_images = torch.stack(six_channel_images).to(device)
                
                for image, label, patient_id in zip(six_channel_images, labels, patient_ids):
                    filtered_rows = data[(data['patientId'] == patient_id) & (data['Target'] == 1)]
                    if not filtered_rows.empty: 
                        image = image[None]
                        output = model(image)
                        #prediction = torch.argmax(output, dim=1)
                        expl = model.explain(image)
                        prediction = expl['prediction']
                        contribution_map = expl['contribution_map'].squeeze(0).cpu()
                        proportion_total = 0.0
                        num_boxes = len(filtered_rows) 
                        for _, row in filtered_rows.iterrows():
                            x, y, width, height = round(row["x"] * scale_x), round(row["y"] * scale_y), round(row["width"] * scale_x), round(row["height"] * scale_y)
                            coordinates_list = [x, y, x + width, y + height]
                            coordinates_tensor = torch.tensor(coordinates_list, dtype=torch.int32)
                            ebpg_result = energy_point_game_recall(coordinates_tensor, contribution_map)
                            proportion_total += ebpg_result
                        
                        average_proportion = proportion_total / num_boxes if num_boxes > 0 else 0.0
                        proportions.append(average_proportion)   
                        if prediction == 1:
                            proportions_correct.append(average_proportion)
                            count_correct = count_correct + 1
                        else:
                            proportions_incorrect.append(average_proportion)
                            count_incorrect = count_incorrect + 1
        if proportions:
            avg_proportion = sum(proportions) / len(proportions)
            avg_proportion_incorrect = sum(proportions_incorrect) / len(proportions_incorrect)
            avg_proportion_correct = sum(proportions_correct) / len(proportions_correct)
            avg_proportions.append(avg_proportion)
            avg_proportions_incorrect.append(avg_proportion_incorrect)
            avg_proportions_correct.append(avg_proportion_correct)

        avg_proportion = round(avg_proportion.item(), 4)
        avg_proportion_incorrect = round(avg_proportion_incorrect.item(), 4)
        avg_proportion_correct = round(avg_proportion_correct.item(), 4)
        
        print(f"Average Energy-Based Pointing Game Proportion (Positive): {avg_proportion}")
        print(f"Average Energy-Based Pointing Game Proportion (Positive) of Incorrectly Classified Images: {avg_proportion_incorrect}, Count: {count_incorrect}", flush=True)
        print(f"Average Energy-Based Pointing Game Proportion (Positive) of Correctly Classified Images: {avg_proportion_correct}, Count: {count_correct}", flush=True)



    final_avg_prop = sum(avg_proportions) / len(avg_proportions) 
    final_avg_prop_incorrect = sum(avg_proportions_incorrect) / len(avg_proportions_incorrect)
    final_avg_prop_correct = sum(avg_proportions_correct) / len(avg_proportions_correct)

    final_avg_prop = round(final_avg_prop.item(), 4)
    final_avg_prop_incorrect = round(final_avg_prop_incorrect.item(), 4)
    final_avg_prop_correct = round(final_avg_prop_correct.item(), 4)
    print(f"Average Energy-Based Pointing Game Proportion (Positive) over all folds: {final_avg_prop}", flush=True)
    print(f"Average Energy-Based Pointing Game Proportion (Positive) of Incorrectly Classified Images over all folds: {final_avg_prop_incorrect}", flush=True)
    print(f"Average Energy-Based Pointing Game Proportion (Positive) of Correctly Classified Images over all folds: {final_avg_prop_correct}", flush=True)

        
        # Store results per fold with seed information
    fold_results = {
        'seed': seed,
        'fold': fold,
        'avg_proportion': final_avg_prop,
        'avg_incorrect': final_avg_prop_incorrect,
        'avg_correct': final_avg_prop_correct,
    }
    all_results.append(fold_results)

# Calculate aggregate statistics
def calculate_stats(values):
    return f"{np.mean(values):.4f} ± {np.std(values):.4f}"

final_proportions = [r['avg_proportion'] for r in all_results]
final_incorrect = [r['avg_incorrect'] for r in all_results]
final_correct = [r['avg_correct'] for r in all_results]

print("\n\n=== Final Aggregate Statistics ===")
print(f"Overall EPG (Positive): {calculate_stats(final_proportions)}")
print(f"EPG Incorrect Classifications: {calculate_stats(final_incorrect)}")
print(f"EPG Correct Classifications: {calculate_stats(final_correct)}")



Processing seed 0



Using cache found in C:\Users\Admin/.cache\torch\hub\B-cos_B-cos-v2_main
C:\Users\Admin/.cache\torch\hub\B-cos_B-cos-v2_main\bcos\common.py:152: UserWarning: Input tensor did not require grad! Has been set automatically to True!
  warnings.warn(


Average Energy-Based Pointing Game Proportion (Positive): 0.9919
Average Energy-Based Pointing Game Proportion (Positive) of Incorrectly Classified Images: 0.9837, Count: 209
Average Energy-Based Pointing Game Proportion (Positive) of Correctly Classified Images: 0.9937, Count: 993


Using cache found in C:\Users\Admin/.cache\torch\hub\B-cos_B-cos-v2_main


Average Energy-Based Pointing Game Proportion (Positive): 0.9907
Average Energy-Based Pointing Game Proportion (Positive) of Incorrectly Classified Images: 0.9885, Count: 314
Average Energy-Based Pointing Game Proportion (Positive) of Correctly Classified Images: 0.9915, Count: 888


...

Average Energy-Based Pointing Game Proportion (Positive): 0.9945
Average Energy-Based Pointing Game Proportion (Positive) of Incorrectly Classified Images: 0.9837, Count: 260
Average Energy-Based Pointing Game Proportion (Positive) of Correctly Classified Images: 0.9974, Count: 943


Using cache found in C:\Users\Admin/.cache\torch\hub\B-cos_B-cos-v2_main


Average Energy-Based Pointing Game Proportion (Positive): 0.0217
Average Energy-Based Pointing Game Proportion (Positive) of Incorrectly Classified Images: 0.0188, Count: 331
Average Energy-Based Pointing Game Proportion (Positive) of Correctly Classified Images: 0.0228, Count: 872


Using cache found in C:\Users\Admin/.cache\torch\hub\B-cos_B-cos-v2_main


Average Energy-Based Pointing Game Proportion (Positive): 0.1533
Average Energy-Based Pointing Game Proportion (Positive) of Incorrectly Classified Images: 0.0063, Count: 251
Average Energy-Based Pointing Game Proportion (Positive) of Correctly Classified Images: 0.1921, Count: 951
Average Energy-Based Pointing Game Proportion (Positive) over all folds: 0.6336
Average Energy-Based Pointing Game Proportion (Positive) of Incorrectly Classified Images over all folds: 0.601
Average Energy-Based Pointing Game Proportion (Positive) of Correctly Classified Images over all folds: 0.6423


=== Final Aggregate Statistics ===
Overall EPG (Positive): 0.6486 ± 0.0150
EPG Incorrect Classifications: 0.6129 ± 0.0119
EPG Correct Classifications: 0.6603 ± 0.0180
